# Model of Coupled Oscillators with one End Free and the other Bound

## Research Question
How to model coupled oscillators moving in a plane with one end fixed and the other free?

## Background

I would like to first acknowledge my professor Dr. Titus for providing the background information to this project.


Defining the ball between the two oscillators as the system and applying the Momentum Principle.

Momentum Principle:

$$\begin{align*}
\frac{d\vec{p}}{dt} & = \vec{F}_{net}\\
\frac{d\vec{p}}{dt} & = \vec{F}_{1} + \vec{F}_2 \\
\frac{d\vec{p}}{dt} & = -k_ss_1\hat{L}_1 + -k_ss_2\hat{L}_2 \\
\frac{d\vec{p}}{dt} & = -k_ss_1\langle 1, 0, 0 \rangle + -k_ss_2\langle -1, 0, 0 \rangle \\
\frac{d\vec{p}}{dt} & = -k_s(L_1-L_0)\langle 1, 0, 0 \rangle + -k_s(L_2-L_0)\langle -1, 0, 0 \rangle \\
%\frac{d\vec{p}}{dt} & = -k_s(L_1-L_0 -L_2 + L_0)\langle 1, 0, 0 \rangle \\
\frac{d\vec{p}}{dt} & = -k_s(L_1 -L_2)\langle 1, 0, 0 \rangle 
\end{align*}$$


The Momentum Principle shows that $d\vec{p}/dt$ is in the x-direction and is proportional to $(L_1 -L_2)$, this being the difference in the lengths of the springs. Since each ball is connected to exactly two springs, when the Momentum Principle is applied to every ball in a line of coupled oscillators, the same result will be accomplished: the rate of change of the momentum of a ball is in the x-direction and is proportional to the difference in lengths of the springs attached to the ball.

To solve for $d\vec{p}/dt$, define a variable $u$ for the x-component of the position of the ball relative to its equilibrium position, so $u = x-x_{equil}$. This is the *relative position* of the ball. Once the ball is positioned to the left, $u$ is negative, the first spring has a length of $L_0 + u$, and the second one has a length of $L_0-u$. The difference in the lengths of the springs is $L_1 -L_2=2u$, and the Momentum Principle gives $dp_x/dt = -2k_su$.



$$\frac{d^2x}{dt^2}=-\frac{k_s}{m}x \ ,\quad \omega = \sqrt{\frac{k_s}{m}}$$

The solution is sinusoidal, $x(t) = A\,\cos(\omega t + \phi)$, with an angular frequency $\omega = \sqrt{k_s/m}$. The phase $\phi$ is determined by the initial conditions $x_0$ and $v_{x0}$. For the ball connected between two springs, the differential equation is:
$$
\begin{align*}
\frac{dp_x}{dt} & = -2k_su \\
\frac{mdv_x}{dt} & = -2k_su \\
\frac{d\kern1pt^2x}{dt\kern1pt^2} & =-\frac{2k_s}{m}u 
\end{align*}
$$

Since the derivative of $u=x-x_{eq}$ is $du/dt = dx/dt$, and the differential equation of the system is

$$
\frac{d^2u}{dt^2} =-\frac{2k_s}{m}u \ ,\quad \omega =  \sqrt{\frac{2k_s}{m}}
$$

Its solution is also sinusoidal,  $u(t) = A\,\cos(\omega t + \phi)$, with an angular frequency $\omega = \sqrt{2k_s/m}$. The ball oscillates with a sinusoidal displacement from equilibrium, but with a frequency $\sqrt{2}$ times larger than if there was only one spring.

## Computational Model
The coupled osccilartors with one end bound and the other end free is modelled below.  It involves 10 objects/balls along the springs, with 11 springs connecting them to a singular wall.

## Assumptions

Assuming that the springs are all initially relaxed perfectly and the initial force only affects the first spring at first.

In [9]:
import numpy as np
import matplotlib.pyplot as plt

#animate with vpython
from vpython import * #watch out for namespace conflicts

# calculate eigenvalues and eigenvectors
from scipy.linalg import eigh

# print to markdown
from IPython.display import display, Markdown, Latex


- Function for fixed end
- Function for free end

The general coupled oscilattors code with both ends fixed was provided by my professor Dr. Titus, my classmate, Nick Palladino, helped edit the code to change it from being both endfs being fixed to only one.

In [10]:
def animate(N):
    
    # properties of the system
    M = 12 #mass of string in kg
    L = 10 #length of string in m
    K = 1 #effective stiffness in N/m

    # properties of each mass
    m = M/N
    L0 = L/(N-1)
    k = K*(N-1)

    # build matrices
    A=np.zeros((N,N))

    for i in range(N):
        A[i,i] = 2*k
        if i==0:
            A[0,i+1]=-k
        elif i==N-1:
            A[i,i-1]=-k
            A[i,i] = k #free end
        else:
            A[i,i-1]=-k
            A[i,i+1]=-k

    B=m*np.identity(N)

    # find eigenvalues and eigenvectors
    lamb, a = eigh(A, B)
    omegas = np.sqrt(lamb)

    ui = np.zeros(N) # initial displacement
    ui[0] = -L0/2 # displace first mass

    # calculate coefficients using initial conditions
    C = np.linalg.solve(a,ui)

    #build the general solution
    h = 0.01
    ttotal = 20
    t = np.arange(0,ttotal,h)

    u = np.zeros((len(t),N+1))
    u[:,0] = np.transpose(t) #store time in first column, u_i in other columns


    # Actual Animation Code
    scene=canvas()

    scene.background=color.white
    scene.fov = 0.1

    thick = L0/20
    height=L0/2
    
  
    wall1 = box(pos= vec(-L/2 - thick, 0, 0), size=vec(thick,height,height), color=vec(0.5,0.5,0.5))
    balls=[]
   
    xballs = np.linspace(-L/2+L0,L/2,N)
  
    springs=[]
    springs.append(helix(pos=vec(-L/2,0,0), axis=vec(L0,0,0), color=color.orange, radius=L0/20))

    i=0
    for x in xballs:
        ball = sphere(pos=vec(x,0,0), radius=L0/10, color=color.red)
        ball.eq = x
        balls.append(ball)
        if i < N-1: #no spring to the right of the last mass
            spring = helix(pos=ball.pos, axis=vec(L0,0,0), color=color.orange, radius=ball.radius/2)
            springs.append(spring)
            i = i + 1

    # set initial positions
    for i in range(0,len(balls)):
        ball = balls[i]
        ball.pos.x = ball.eq + u[0,i+1]
   
    # all springs except first
    for i in range(1, len(springs)): 
        spring = springs[i]
        spring.pos=balls[i-1].pos
        spring.axis=balls[i].pos-spring.pos

    # update first spring
    springs[0].axis = balls[0].pos-springs[0].pos
   
    # animate
    scene.pause()
    t=0
    for n in range(len(u[:,0])):
        rate(200)
        for i in range(0,len(balls)):
            ball = balls[i]
            ball.pos.x = ball.eq + u[n,i+1]
        
        for i in range(1, len(springs)): #all springs except first 
            spring = springs[i]
            spring.pos=balls[i-1].pos
            spring.axis=balls[i].pos-spring.pos

        #update first spring
        springs[0].axis = balls[0].pos-springs[0].pos
        
    return 

In [ ]:
animate(10)

<IPython.core.display.Javascript object>

## Conclusion

The simulation can now model coupled oscillators moving back and forth in a one dimensional plane without being bound at both ends.  The model can now move freely at one end and fixed at the other.  In the future, it would be interesting to look at both ends being free and taking into account translational velocity as well.